In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ugo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ugo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# chargement des données
steam = pd.read_csv('archive/steam.csv')
steam_description_data = pd.read_csv('archive/steam_description_data.csv')

# on ne devrait pas avoir besoin de celles-ci
'''
steam_media_data = pd.read_csv('steam_media_data.csv')
steam_requirements_data = pd.read_csv('steam_requirements_data.csv')
steam_support_info = pd.read_csv('steam_support_info.csv')
steamspy_tag_data = pd.read_csv('steamspy_tag_data.csv')
'''

# fonction pour afficher les informations d'un fichier
# si dimensions_type = True, les types des dimensions sont également affichés
def info(file, *dimensions_type):
    dataset_panda = pd.read_csv(file)
    print('*****'+file+'*****\n'+'Nombre de donnée: ', dataset_panda.shape[0],'\nNombre de dimension: ',
          dataset_panda.shape[1],'\n')
    if(dimensions_type):
        print('Types des dimensions:\n', dataset_panda.dtypes)

In [3]:
info('archive/steam.csv', True)
steam

*****archive/steam.csv*****
Nombre de donnée:  27075 
Nombre de dimension:  18 

Types des dimensions:
 appid                 int64
name                 object
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
dtype: object


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19


In [4]:
info('archive/steam_description_data.csv', True)
steam_description_data

*****archive/steam_description_data.csv*****
Nombre de donnée:  27334 
Nombre de dimension:  4 

Types des dimensions:
 steam_appid              int64
detailed_description    object
about_the_game          object
short_description       object
dtype: object


,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...
...,...,...,...,...
27329,1065230,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",The Room of Pandora is a third-person interact...
27330,1065570,Have you ever been so lonely that no one but y...,Have you ever been so lonely that no one but y...,Cyber Gun is a hardcore first-person shooter w...
27331,1065650,<strong>Super Star Blast </strong>is a space b...,<strong>Super Star Blast </strong>is a space b...,Super Star Blast is a space based game with ch...
27332,1066700,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...,Pursue a snow-white deer through an enchanted ...


## Preprocessing

- retirer les majuscules : fait
- retirer les liens
- retirer code HTML : fait
- retirer les différents types de parenthèses : fait
- retirer la ponctuation : fait
- retirer certains mots (nom du jeu?)
- verifier si toutes les valeurs existent dans X/y
- retirer caractères spéciaux : fait <br>
- SUPPRIMER LES DESCRIPTIONS QUI NE SONT PAS EN ANGLAIS : 日本語バージョンダウンロード開始されました。X[4569] !!! 20124
Plus d'infos: https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

In [5]:
# supprimer des caractères d'un string:
# nomString = nomString.replace(caractere_a_remplacer, caractere_de_remplacement)
'''
liste_de_caractere_a_supprimer = []
string_sur_lequel_applique_le_filtre = 'salut cest ugo'
translation = string_sur_lequel_applique_le_filtre.maketrans({i:"" for i in liste_de_caractere_a_supprimer}) 
#applying the table on the string using the translate() 
string_sur_lequel_applique_le_filtre = string_sur_lequel_applique_le_filtre.translate(translation)

Plus d'infos:
https://www.pythonpool.com/remove-character-from-string-python/#:~:text=%207%20Ways%20to%20Remove%20Character%20From%20String,is%20a%20quite%20common%20way%20%28though...%20More%20
'''

'\nliste_de_caractere_a_supprimer = []\nstring_sur_lequel_applique_le_filtre = \'salut cest ugo\'\ntranslation = string_sur_lequel_applique_le_filtre.maketrans({i:"" for i in liste_de_caractere_a_supprimer}) \n#applying the table on the string using the translate() \nstring_sur_lequel_applique_le_filtre = string_sur_lequel_applique_le_filtre.translate(translation)\n\nPlus d\'infos:\nhttps://www.pythonpool.com/remove-character-from-string-python/#:~:text=%207%20Ways%20to%20Remove%20Character%20From%20String,is%20a%20quite%20common%20way%20%28though...%20More%20\n'

#### Récupération des données

In [6]:
# On récupère les valeurs de la colonne steam_appid du dataset steam
steam_id = steam[steam.columns[0]].values
# On récupère les valeurs de la colonne steam_appid du dataset steam_description_data
steam_description_id = steam_description_data[steam_description_data.columns[0]].values

# Il y a plus d'éléments dans steam_description_data que dans steam
steam_id, steam_description_id,len(steam_id), len(steam_description_id)

(array([     10,      20,      30, ..., 1065650, 1066700, 1069460],
       dtype=int64),
 array([     10,      20,      30, ..., 1065650, 1066700, 1069460],
       dtype=int64),
 27075,
 27334)

In [7]:
# En recuperant les éléments en commun dans les deux datasets on voit que
# tous les éléments de steam sont dans steam_description_data
final_id = np.intersect1d(steam_id, steam_description_id)
final_id == steam_id

array([ True,  True,  True, ...,  True,  True,  True])

In [8]:
# On récupère toutes les valeurs de toutes les colonnes du fichier steam_description_data.csv
description = steam_description_data[steam_description_data.columns].values

features = []
# Le fichier steam_description_data.csv contient plus de jeux que steam.csv, on ne récupère que ceux qu'ils ont en commun
for i in range(len(description)):
    if(description[i,0] in steam_id):  
        features.append(description[i, 1:])

# On récupère les données
X = np.array(features)
# On récupère les labels
label = steam.columns[9] # la colonne 9 correspond au genre du jeu
y = steam[label].values

In [9]:
# supprime de la liste X les caractères présents dans la liste characters
def delete_character(X, y, characters):
    N = X.shape[0]
    D = X.shape[1]
    # les mots commençant par ces caractères seront supprimés
    starter_to_delete = ('http', 'www', '/','"http','"www')
    # les éléments de characters seront remplacés par un espace " "
    translation = str.maketrans({i:" " for i in characters})
    for i in range(N):
        # certains labels contiennent le symbole '&', on le supprime
        y[i] = y[i].translate(translation)
        for j in range(D):
            # on sépare les mots et on les met dans la liste X_split
            X_split = X[i,j].split(' ')
            X_without_link = ""
            for k in range(len(X_split)):
                # on vérifie que chaque mot ne commence pas par un des starters à supprimer
                if(not X_split[k].startswith(starter_to_delete)):
                    # s'il s'agit du dernier élément on ne met pas d'espace après
                    if(k == len(X_split) - 1): 
                        X_without_link += X_split[k]
                    else:
                        X_without_link += X_split[k] + " "
            # on supprime les caractères à supprimer
            X[i,j] = X_without_link.translate(translation)

### Suppression de texte: HTLM, brackets...

In [10]:
example = "Voici comment on supprime<br>le code HTLM</br><span></span><li><br></li><li>du</li><strong>texte pour<img http//ww.dkjfdjsf> pouvoir<li>l'utiliser<br> dans</br> le projet<ul> d'IAS !</ul>"
print('Texte avec du code HTML: \n', example, '\n')

# on split le texte avec la balise ouvrante
split_example = example.split('<')
print('1er split:\n',split_example,'\n')

# on le split à nouveau avec la balise fermante
example_without_HTML = ""

# supprimer html
print('2e split:\n')
for i in range(len(split_example)):
    split_example[i] = split_example[i].split('>')
    print(split_example[i])
    if(i == 0):
        example_without_HTML += split_example[i][0]
    elif(len(split_example[i]) > 1 and split_example[i][1] != ""):
        if(split_example[i][1][0] == " "):
            example_without_HTML += split_example[i][1]
        else:
            example_without_HTML += " " + split_example[i][1]

print("\nOn concatène le premier tableau avec le deuxième élément de tous les autres et on obtient: \n\n", example_without_HTML)

Texte avec du code HTML: 
 Voici comment on supprime<br>le code HTLM</br><span></span><li><br></li><li>du</li><strong>texte pour<img http//ww.dkjfdjsf> pouvoir<li>l'utiliser<br> dans</br> le projet<ul> d'IAS !</ul> 

1er split:
 ['Voici comment on supprime', 'br>le code HTLM', '/br>', 'span>', '/span>', 'li>', 'br>', '/li>', 'li>du', '/li>', 'strong>texte pour', 'img http//ww.dkjfdjsf> pouvoir', "li>l'utiliser", 'br> dans', '/br> le projet', "ul> d'IAS !", '/ul>'] 

2e split:

['Voici comment on supprime']
['br', 'le code HTLM']
['/br', '']
['span', '']
['/span', '']
['li', '']
['br', '']
['/li', '']
['li', 'du']
['/li', '']
['strong', 'texte pour']
['img http//ww.dkjfdjsf', ' pouvoir']
['li', "l'utiliser"]
['br', ' dans']
['/br', ' le projet']
['ul', " d'IAS !"]
['/ul', '']

On concatène le premier tableau avec le deuxième élément de tous les autres et on obtient: 

 Voici comment on supprime le code HTLM du texte pour pouvoir l'utiliser dans le projet d'IAS !


In [11]:
# supprime le code HTML des données, pour ça on supprime les balises et tous les caractères qui se trouvent entre des balises
def delete_HTML(X):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            # on split le texte avec le caractère '<'
            X_split = X[i,j].split('<')
            # correspond au texte filtré
            X_without_HTML = ""
            for k in range(len(X_split)):
                # on split le texte dèjà séparé avec le caractère '>' cette fois
                X_split[k] = X_split[k].split('>')
                if(k == 0): # concerne tous les caractères qui apparaissent avant la premiere balise du texte
                    X_without_HTML += X_split[k][0] # on ne met pas d'espace devant
                elif(len(X_split[k]) > 1 and X_split[k][1] != ''):
                    # si le texte commençait par un espace on en ajoute pas sinon si
                    if(X_split[k][1][0] == ' '):
                        X_without_HTML += X_split[k][1]
                    else:
                        X_without_HTML += " " + X_split[k][1]
            # on remplace le texte contenant du code HTML par le texte filtré
            X[i,j] = X_without_HTML        

In [12]:
# supprime les différents types de parenthèses et leur contenu, on utilise le même principe que pour le filtrage du code HTML
def delete_bracket(X):
    N = X.shape[0]
    D = X.shape[1]
    brackets = np.array([['(', ')'], ['[', ']'], ['{', '}']])
    for b in range(len(brackets)):    
        for i in range(N):
            for j in range(D):
                X_split = X[i,j].split(brackets[b,0])
                X_without_brackets = ""
                for k in range(len(X_split)):
                    X_split[k] = X_split[k].split(brackets[b,1])
                    if(k == 0):
                        X_without_brackets += X_split[k][0]
                    elif(len(X_split[k]) > 1 and X_split[k][1] != ''):
                        if(X_split[k][1][0] == ' '):
                            X_without_brackets += X_split[k][1]
                        else:
                            X_without_brackets += " " + X_split[k][1]
                X[i,j] = X_without_brackets         

In [13]:
def replace(X, new_values):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            for r in range(len(new_values)):
                X[i,j] = X[i,j].replace(new_values[r][0], new_values[r][1])

In [14]:
def delete_backslash_slash(X):
    N = X.shape[0]
    D = X.shape[1]
    l = ['\x91','\x92','\x94','\x93','\x97','\x96','\x80','\x99','\x98','\xad','\x7f','\u200b','\u200f','\ufeff','/','\n',
        '\t','\r']
    for i in range(N):
        for j in range(D):
            for k in l:
                X[i,j] = X[i,j].replace(k, " ")

In [15]:
def delete_no_english(X, y):
    steam_langage = steam[steam.columns[3]].values
    cpt = 0
    new_X = []
    new_y = []
    N = X.shape[0]
    for i in range(N):
        # si le jeu n'est pas en anglais
        if(steam_langage[i] == 1):
            new_X.append(X[i])
            new_y.append(y[i])
        else:
            cpt += 1
    print(cpt," jeux ont été enlevés\n")
    return np.array(new_X), np.array(new_y)

In [16]:
def delete_no_ascii(X,y):
    new_X = []
    new_y = []
    cpt=0
    for i in range(len(X)):
        # indique si il y a des caractères non ascii
        no_ascii = True
        for e in X[i]:
            # si le caractères n'est pas ascii
            if(re.sub('[ -~]', '', e) != ""):
                no_ascii = False
        if(no_ascii):
            new_X.append(X[i])
            new_y.append(y[i])
        else:
            cpt += 1
    print(cpt," jeux ont été enlevés\n")
    return np.array(new_X), np.array(new_y)

In [17]:
def upperCase_to_lowerCase(X, y):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            X[i,j] = X[i,j].lower()
        y[i] = y[i].lower()

In [18]:
def delete_big_spaces(X, y):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        y[i] = " ".join(y[i].split())
        for j in range(D):
            X[i,j] = " ".join(X[i,j].split())

In [19]:
def delete_numbers(X):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            X_number_convert = ""
            for k in range(len(X[i,j])):
                if(X[i,j][k].isdigit()):
                    X_number_convert += " "
                else:
                    X_number_convert += X[i,j][k]
            X[i,j] = X_number_convert

In [20]:
def delete_stopwords(X, stopwords):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            X_without_stopwords = ""
            X_split = X[i,j].split(" ")
            for k in X_split:
                if(k not in stopwords):
                    X_without_stopwords += k + " "
            X[i,j] = X_without_stopwords

In [21]:
def delete_oneLetter_words(X):
    N = X.shape[0]
    D = X.shape[1]
    for i in range(N):
        for j in range(D):
            X_without_oneLetter_word = ""
            X_split = X[i,j].split(" ")
            for k in X_split:
                if(len(k) > 1):
                    X_without_oneLetter_word += k + " "
            X[i,j] = X_without_oneLetter_word

In [25]:
remplacement = [["ﬁ","fi"],["ê","e"],["é","e"],["è","e"],["ë","e"],["à","a"],["ä","a"],["ä","a"],["å","a"],["ł","l"],["ü","u"]
                ,["û","u"],["ö","o"],["ô","o"],["Ö","o"],["ò","o"],["í","i"],["ï","i"],["î","i"],["Ã","a"],["Ä","a"],["ž","z"]
                ,["ì","i"],["Ñ","n"],["ñ","n"],["á","a"],["ø","o"],["£","e"],["Á","a"],["á","a"],["Ü","u"],["ō","o"],["ð","d"]
                ,["ú","u"],["ã","a"],["É","e"],["×","x"],["Ó","o"],["С","c"],["、",","],["ʃ","sh"],["ɑ","a"],["ñ","n"],["ũ","u"]
                ,["ó","o"],["â","a"],["Û","u"],["ū","u"],["õ","o"],["，",","],["ń","n"],["ﬂ","fl"],["х","x"],["А","a"]
                ,["с","c"],["Ś","s"],["Т","t"],["a","a"]]

In [26]:
character_to_delete = np.array(['•','€','™','*','-','–','¢','˜','«','»','®','+','µ','·','¹','¬','º','½','±','©','§','ª','ƒ',
                              '¿','Ð','¸','¼','Ø','¨','æ','‡','“','”','ç','¥','œ','´','#','—','‘','…', '²', '●', '³', '・',
                                '∞','√','∼','✢','※','◾','■','★','➠','―','„','✓','▶','◆','ລ','▪','♪','♞','♥','Ω','α',
                                'Λ','♫','「','」','：','◕','_','つ','༽','༼','⚠','✔','』','『','►','☆','℗','✗','✪','†','《',
                                '》','一','。','～','‐','″','↑','@','◉','♂','♀','＊','【','】','＊','＜','＞','→','↓','←',
                                '①','②','③','▽','△','○','Ⅶ','□','（','）','%','Ⅱ','~','°','&',',', '!', '?', ':', '.', ';',
                                '"',"'",'′','’','＋'])

In [27]:
stopwords = np.loadtxt("stopwords.txt",str)

In [28]:
X, y = delete_no_english(X, y)

511  jeux ont été enlevés



In [29]:
replace(X, remplacement)

In [30]:
delete_character(X, y, character_to_delete)

In [31]:
delete_numbers(X)

In [32]:
delete_HTML(X)

In [33]:
delete_bracket(X)

In [34]:
delete_backslash_slash(X)

In [35]:
upperCase_to_lowerCase(X, y)

In [36]:
X, y = delete_no_ascii(X, y)

831  jeux ont été enlevés



In [37]:
delete_stopwords(X, stopwords)

In [38]:
delete_oneLetter_words(X)

In [39]:
delete_big_spaces(X, y)

In [38]:
# toutes ces fonctions prennent 3m pour sexecuter

In [39]:
# tester pour les autres caracteres speciaux du style u tilde...
'''
for i in range(len(X)):
    for j in range(3):
        for k in range(len(X[i,j])):
            if(X[i,j][k] in ['Ä', 'Ñ', 'ì', 'ž']):
                print(X[i,j][k], '\n')
'''

"\nfor i in range(len(X)):\n    for j in range(3):\n        for k in range(len(X[i,j])):\n            if(X[i,j][k] in ['Ä', 'Ñ', 'ì', 'ž']):\n                print(X[i,j][k], '\n')\n"

In [40]:
# tester pour les autres caracteres speciaux du style u tilde...
'''
non_ascii_char = []
for i in range(len(X)):
    element = X[i]
    for e in element:
        if(re.sub('[ -~]', '', e) != ""):
            e_split = e.split(' ')
            for l in e_split:
                split2 = l.split()
                for m in split2:
                    if(re.sub('[ -~]', '', m) != ""):
                        non_ascii_char.append(m)
'''

'\nnon_ascii_char = []\nfor i in range(len(X)):\n    element = X[i]\n    for e in element:\n        if(re.sub(\'[ -~]\', \'\', e) != ""):\n            e_split = e.split(\' \')\n            for l in e_split:\n                split2 = l.split()\n                for m in split2:\n                    if(re.sub(\'[ -~]\', \'\', m) != ""):\n                        non_ascii_char.append(m)\n'

In [41]:
'''
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ' '
''' 
# https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#wordnetlemmatizer

"\ndef get_wordnet_pos(treebank_tag):\n\n    if treebank_tag.startswith('J'):\n        return wordnet.ADJ\n    elif treebank_tag.startswith('V'):\n        return wordnet.VERB\n    elif treebank_tag.startswith('N'):\n        return wordnet.NOUN\n    elif treebank_tag.startswith('R'):\n        return wordnet.ADV\n    else:\n        return ' '\n"

In [40]:
def lemmatize(X, split_words=False):
    N = X.shape[0]
    D = X.shape[1]
    new_X = []
    lemmatizer = WordNetLemmatizer()
    for i in range(N):
        new_X.append([])
        for j in range(D):
            X_split = X[i,j].split(" ")  
            X_lemmatized = ""
            for k in range(len(X_split)):
                if(k == len(X_split) - 1):
                    X_lemmatized += lemmatizer.lemmatize(X_split[k], wordnet.VERB)
                else:
                    X_lemmatized += lemmatizer.lemmatize(X_split[k], wordnet.VERB) + " "
            if(split_words):
                new_X[i].append(X_lemmatized.split(" "))
            else:
                new_X[i].append(X_lemmatized)
    return np.array(new_X)

In [43]:
def copy(X):
    new = []
    for i in range(len(X)):
        new.append(X[i])
    return np.array(new)

In [68]:
X_save = copy(X)
X_save[9896,0]

'fate hero hands stranger planet occasion return finally stays decides local people'

In [41]:
X_split = lemmatize(X, split_words=True)

In [42]:
X_no_split = lemmatize(X)

MemoryError: Unable to allocate 5.20 GiB for an array with shape (25733, 3) and data type <U18077

In [70]:
print(X_split[9896,0])

['fate', 'hero', 'hand', 'stranger', 'planet', 'occasion', 'return', 'finally', 'stay', 'decide', 'local', 'people']


In [67]:
X_no_split[9896,0]

'fate hero hand stranger planet occasion return finally stay decide local people'

## Labels

In [53]:
def rename_labels(y):
    N = y.shape[0]
    for i in range(N):
        y[i] = y[i].replace("free to play", "free")
        y[i] = y[i].replace("animation & modeling", "animation")
        y[i] = y[i].replace("video production", "video")
        y[i] = y[i].replace("massively multiplayer", "multiplayer")
        y[i] = y[i].replace("early access", "early")
        y[i] = y[i].replace("design & illustration", "design")
        y[i] = y[i].replace("web publishing", "web")
        y[i] = y[i].replace("software training", "software")
        y[i] = y[i].replace("sexual content", "sexual")
        y[i] = y[i].replace("audio production", "audio")
        y[i] = y[i].replace("game development", "development")
        y[i] = y[i].replace("photo editing", "photo")
        #y[i] = y[i].replace("rpg", "role")

In [54]:
def keep_first_label_only(y):
    N = y.shape[0]
    new_y = []
    for i in range(N):
        y_split = y[i].split(" ")
        new_y.append(y_split[0])
    return np.array(new_y)

In [52]:
y_save = copy(y)
# renommage des labels avant suppression de & -> suppr des caractères !!!
rename_labels(y)
for i in y:
    print(i)

action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action free
action
action
action free strategy
action adventure
action
action free
indie
action
action
action
indie strategy
indie strategy
indie strategy
indie
strategy
strategy
strategy
strategy
strategy
strategy
rpg
animation modeling video
strategy
rpg
action rpg
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action indie rpg
action
action
casual indie
action
action
action
action
action
action
action
action indie strategy
action
strategy
strategy
action simulation strategy
simulation strategy
simulation strategy
action simulation
strategy
strategy
strategy
racing
racing
action
racing
strategy
rpg strategy
strategy
adventure casual
action
casual
casual
casual
casual
casual
casual
casual
casual
casual
casual
strategy casual
casual
casual
strategy
casual
casual
action casual
action
action
strategy
action

action indie
action simulation
adventure indie
action adventure
action adventure rpg
indie rpg strategy
action adventure rpg
action
indie simulation strategy
adventure indie rpg
casual indie
action adventure
action indie racing
action indie simulation
indie strategy
strategy
action indie rpg simulation strategy
action indie rpg
action adventure
action adventure
action indie
action indie simulation
adventure casual indie
action adventure indie
casual free rpg
casual
action adventure indie
action indie
action indie racing
adventure indie rpg strategy
action adventure
education software utilities web
action adventure indie
adventure casual indie simulation
adventure casual indie
action indie
action adventure
casual indie strategy
indie strategy
action free multiplayer rpg
casual indie rpg strategy
strategy
adventure free indie multiplayer rpg strategy
action
indie strategy
action indie
action adventure
casual indie strategy
indie strategy
action indie strategy
indie
action indie
web
casua

action indie
indie
simulation strategy early
action indie rpg
action indie
adventure casual indie
adventure casual indie rpg
action casual indie
casual indie rpg simulation
action indie simulation strategy
indie simulation strategy
adventure indie
action indie
adventure indie rpg simulation strategy
action adventure casual indie rpg
action adventure indie
indie simulation strategy
action adventure indie
free indie strategy
adventure indie
action casual indie
action adventure casual
indie rpg strategy
adventure
adventure indie
action adventure indie
adventure casual
adventure casual
adventure casual
adventure casual
adventure casual
action strategy
adventure casual
adventure casual
adventure casual
adventure casual
adventure casual
adventure casual
adventure casual
casual
casual simulation strategy
action indie strategy
action indie strategy
action
simulation strategy
action rpg strategy
adventure indie
free multiplayer rpg
indie rpg simulation strategy
action adventure indie
indie stra

action
indie rpg
action adventure indie rpg simulation strategy
casual indie strategy
action casual
action casual
action casual indie multiplayer rpg early
indie simulation strategy early
action rpg
action simulation early
casual indie simulation
indie strategy
indie strategy
indie
casual indie
indie
action indie
adventure casual indie
adventure casual indie
adventure indie rpg
adventure indie rpg strategy
casual indie simulation
action free indie
casual indie strategy
adventure casual
action indie
indie rpg
racing sports early
action indie rpg strategy
indie
action casual free indie strategy
adventure casual indie
action casual indie
indie
adventure casual rpg strategy
action indie
action adventure free multiplayer
action
casual indie simulation strategy early
action indie
indie rpg strategy
action casual indie
action casual indie early
casual indie
adventure casual indie strategy
casual indie simulation
action indie
action casual indie
indie
action adventure casual free indie multipl

action
free multiplayer strategy
indie simulation
indie simulation strategy
adventure casual indie rpg
indie strategy
action indie
sexual action
indie strategy
adventure casual indie
action adventure indie simulation
indie strategy
action indie simulation
action casual indie
sexual nudity casual indie
adventure casual indie
indie simulation
action adventure rpg
action adventure indie
action adventure indie simulation
adventure indie rpg simulation strategy
indie
action casual indie
action indie early
action indie rpg strategy
casual indie strategy
action adventure indie rpg
action
action rpg
indie simulation strategy
action indie strategy
indie simulation strategy
free strategy
adventure casual indie
adventure
adventure free indie rpg
action casual indie strategy
action adventure indie
casual indie
indie
action free indie
action adventure indie simulation early
casual indie strategy
casual indie
indie strategy
action indie strategy
adventure casual
casual indie
adventure
action indie s

...

simulation
simulation
casual indie
casual indie simulation
casual indie
action casual free indie racing
action casual indie racing simulation sports strategy early
sexual nudity adventure casual indie rpg
indie
adventure indie rpg
action casual indie
action casual indie
action indie
indie rpg simulation strategy
action adventure casual indie
action adventure indie rpg
casual indie
casual indie
action casual indie
action adventure casual
casual indie
casual indie simulation
adventure indie
casual strategy
casual free simulation
action adventure indie
simulation strategy early
action casual indie sports
action indie
adventure casual indie rpg
simulation
action adventure indie
action indie rpg early
action adventure indie rpg
adventure
casual indie rpg simulation strategy
action adventure casual free indie racing rpg simulation sports
casual
adventure indie simulation
action indie
action casual free indie simulation
adventure casual indie rpg strategy
violent action adventure indie
action

casual indie simulation
adventure casual indie rpg strategy
action casual indie early
casual indie
adventure casual indie
action casual
indie
adventure casual indie
action adventure indie
action casual
adventure indie rpg
violent action casual indie early
action casual simulation strategy
action indie sports
action indie early
action indie
adventure casual indie simulation accounting
indie strategy
adventure indie
casual indie simulation strategy
action multiplayer early
action adventure indie
action adventure indie
action adventure indie
rpg
action
adventure indie rpg
casual indie
adventure indie rpg
indie
casual
action adventure rpg
casual indie strategy
free sports strategy
action indie
casual indie racing sports
rpg
casual
casual
adventure casual indie
casual indie simulation sports
violent gore action adventure casual sports
casual indie simulation
action adventure indie
action casual indie rpg simulation sports strategy
adventure casual indie
action casual simulation early
casual

action indie strategy
adventure simulation
adventure casual indie simulation
action adventure
action casual indie
adventure free indie
indie simulation
casual indie simulation sports
action indie racing sports
action
sexual nudity casual indie simulation
action adventure casual free indie
adventure casual indie rpg strategy
indie simulation early
action adventure indie
adventure free indie
action adventure casual indie racing simulation
action free indie sports early
indie rpg
action adventure indie rpg
casual indie
sexual nudity
casual indie rpg simulation strategy early
casual racing simulation
adventure indie early
action adventure indie
casual indie strategy
casual indie strategy
casual indie
casual indie simulation strategy
action indie early
adventure indie rpg early
casual indie strategy
action casual indie
action adventure indie simulation
action indie rpg strategy early
casual indie simulation
action indie
rpg simulation sports
simulation
action simulation early
violent gore c

adventure
adventure indie
indie
action casual free indie early
adventure casual
adventure
casual indie
casual indie
action indie
strategy
simulation
strategy
strategy
strategy
action adventure casual indie rpg
action casual indie
adventure free simulation strategy
action
utilities
indie
action indie early
rpg
action adventure casual indie
casual indie sports early
adventure
indie
indie audio utilities
action adventure
action casual indie strategy
adventure casual free sports
casual strategy
indie early
adventure casual indie
action indie
casual
adventure casual indie strategy
indie early
casual indie simulation
adventure casual indie simulation
adventure casual
action casual indie strategy
adventure
action adventure casual indie strategy
adventure casual free rpg
action casual indie
action indie racing
indie early
casual indie
casual indie
adventure indie
sports
casual
action adventure casual indie strategy
animation modeling design illustration utilities
action adventure casual indie


adventure indie
action free indie multiplayer early
action adventure indie multiplayer rpg strategy early
indie simulation strategy early
indie
action free strategy
rpg
action adventure casual free indie multiplayer racing rpg simulation sports strategy
racing
adventure indie rpg
action casual indie
casual
casual indie
casual sports
casual indie simulation
action casual indie rpg strategy
action casual indie
action adventure indie
action free indie
action adventure casual indie
action adventure casual free indie rpg simulation sports strategy
action adventure indie simulation
casual indie
action adventure indie
indie sports strategy
casual indie
action adventure indie racing simulation
adventure indie
action indie early
action adventure indie
indie strategy
adventure free indie
indie
casual
casual indie
casual
casual indie simulation sports strategy
action adventure casual indie simulation
indie rpg
action free
simulation
adventure casual indie simulation strategy
action
casual indie
a

In [80]:
y_first_label = keep_first_label_only(y)

On voit dans la cellule suivante que plusieurs jeux ont les mêmes "couples" de genre, on peut donc associer un entier à chacun de ces couples, et chaque entier devient un nouveau label

In [81]:
y_couple = []
for i in y_save:
    if(i not in y_couple):
        y_couple.append(i)
    else:
        print(i)

action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action free
action
action
action
indie strategy
indie strategy
indie
strategy
strategy
strategy
strategy
strategy
strategy
rpg
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
action
strategy
strategy
simulation strategy
strategy
strategy
strategy
racing
action
racing
strategy
strategy
action
casual
casual
casual
casual
casual
casual
casual
casual
casual
casual
casual
strategy
casual
casual
action
action
strategy
action
action
action
action
action
strategy
strategy
rpg
strategy
strategy
strategy
racing
racing
rpg
simulation strategy
rpg
action rpg
action strategy
strategy
strategy
strategy
action
strategy
strategy
strategy
strategy
strategy
strategy
strategy
strategy
casual indie
action indie strategy
action
action
action
adventure

action adventure indie
adventure indie
action indie
adventure indie rpg
adventure casual
adventure indie
action indie
action indie rpg strategy
indie strategy
action indie
casual strategy
action rpg
adventure indie
action adventure indie
action adventure indie rpg
action
action indie rpg strategy
adventure rpg
action free indie
adventure indie
adventure indie
action adventure rpg
action indie
action free indie multiplayer
adventure indie
casual indie simulation strategy
action adventure indie
adventure casual indie rpg strategy
casual indie strategy
casual indie simulation
adventure indie
rpg strategy
indie rpg
casual indie
action adventure indie
action indie
action indie rpg
action adventure indie rpg early
indie rpg
indie rpg
action indie
simulation
action indie
casual indie simulation strategy
adventure indie rpg
action indie rpg
simulation strategy
adventure
casual indie
utilities
action indie rpg strategy
casual indie strategy
action
action adventure
simulation
simulation
rpg
indi

action indie
action casual indie early
adventure indie early
indie
action indie strategy
casual strategy
indie
action adventure casual indie
adventure casual
action adventure indie rpg simulation
action casual indie racing simulation
action casual indie
indie rpg strategy
casual strategy
indie simulation early
action casual indie
indie multiplayer rpg early
casual indie simulation strategy
action free indie rpg
adventure casual indie
adventure
action indie
simulation
indie simulation
action adventure casual indie rpg strategy
action casual indie racing simulation sports
adventure racing
action adventure casual indie rpg
action indie
casual
casual indie
adventure indie
action indie rpg simulation strategy
action indie rpg
action indie
casual indie strategy
indie rpg
casual
indie
adventure casual
action casual indie racing simulation
indie
casual indie
action
action casual free indie
indie rpg
casual indie
adventure indie
adventure indie
adventure indie
adventure indie
adventure indie
ad

casual indie strategy
indie rpg strategy
action adventure casual indie simulation
casual indie
adventure indie
casual indie
action indie simulation early
casual free indie rpg simulation strategy
action free indie strategy
action adventure casual indie rpg simulation strategy
action adventure indie simulation strategy
action indie simulation
action indie
indie
action adventure multiplayer simulation early
indie rpg strategy early
action adventure
action indie
indie simulation
adventure casual indie
indie simulation strategy
action adventure indie early
adventure indie
action indie
adventure casual strategy
adventure casual strategy
action
adventure casual simulation strategy
indie racing sports
adventure indie simulation
adventure casual
action indie
action indie early
indie strategy
action indie racing
action indie
action adventure casual indie simulation strategy
action adventure indie rpg
action adventure indie
casual indie simulation
indie rpg
action indie strategy
indie
action ind

simulation sports
action casual indie racing simulation sports
indie strategy early
utilities
adventure indie
action adventure indie rpg
indie simulation
violent gore action indie
free indie
violent action adventure indie
casual indie simulation early
adventure indie strategy
casual indie simulation
adventure casual indie early
action free early
action indie
strategy
nudity violent gore action adventure
action adventure casual indie
adventure indie
action casual indie strategy
indie
action indie rpg strategy
action indie rpg
action adventure indie rpg
casual indie
action adventure indie
adventure
adventure
action adventure casual
adventure indie
casual indie
adventure casual indie
action casual free indie
indie rpg
casual
strategy
casual indie
casual indie rpg strategy
action casual indie
strategy
action casual indie strategy
action simulation
action simulation
simulation
action simulation
simulation
animation modeling design illustration utilities
action simulation
indie simulation st

...

casual early
action indie
indie racing
indie simulation
indie strategy
adventure casual indie rpg simulation early
action adventure rpg early
adventure casual indie strategy
action adventure indie
adventure indie
indie rpg
casual indie rpg simulation strategy
action simulation
violent gore action adventure
adventure casual indie rpg simulation
casual indie
action casual free indie
action adventure casual indie sports
adventure indie
action adventure indie
violent gore action adventure casual indie
action indie
action casual indie
indie simulation strategy
action indie early
indie simulation strategy
action indie sports early
action adventure casual indie rpg simulation strategy
casual indie simulation
violent action adventure indie rpg
action adventure indie
adventure strategy
indie rpg strategy early
action adventure
indie
adventure casual indie simulation sports strategy
action casual indie simulation strategy
adventure casual indie
adventure free multiplayer rpg
action adventure ind

casual indie simulation
casual indie strategy
adventure indie
indie simulation strategy
adventure casual indie simulation
casual indie strategy
indie simulation early
indie
casual simulation
adventure indie
indie
action indie early
casual indie strategy
action adventure casual indie
adventure casual
action adventure indie
action adventure indie rpg
casual indie simulation
action adventure indie
casual free indie simulation strategy
casual free indie
indie multiplayer rpg
indie racing sports
indie early
adventure casual indie
indie rpg
indie
indie
adventure indie rpg
adventure indie rpg early
free multiplayer rpg strategy early
simulation strategy
racing simulation sports
violent gore action indie simulation
indie simulation strategy
indie rpg
action adventure indie rpg early
nudity violent gore action adventure indie
adventure indie
action indie
action casual indie
action indie racing sports
action indie
action adventure indie
action adventure indie early
action indie simulation
action

action adventure indie strategy
adventure casual indie racing
adventure casual indie
adventure
casual indie
action indie
action indie
casual indie
indie
casual indie early
action indie
adventure free indie simulation
adventure casual indie simulation
casual indie
action indie
action adventure indie rpg early
action adventure casual indie simulation
adventure casual
adventure casual
adventure casual
action indie strategy
violent gore adventure indie
free
action adventure casual indie racing simulation sports
adventure casual
indie multiplayer rpg strategy
casual indie
action
action adventure indie rpg early
adventure casual
adventure casual
adventure casual
adventure casual
casual free indie simulation
action free indie
action indie strategy
adventure indie
action indie
casual indie simulation strategy
action indie
adventure casual indie
action racing
casual indie
adventure casual indie
action indie rpg early
action casual indie
action adventure indie rpg early
casual indie
adventure in

action indie simulation sports
violent action adventure indie
casual indie simulation sports
casual
casual indie
casual indie rpg simulation early
action adventure free indie early
action adventure casual
casual free indie rpg simulation
action casual indie
action adventure casual indie
casual indie strategy
adventure indie
free strategy
action indie
action casual indie
free indie
casual
action adventure indie simulation early
adventure casual indie simulation strategy early
action
casual indie
action adventure casual indie
action indie
indie rpg simulation strategy
rpg
casual indie
adventure
casual indie
indie
action casual indie simulation
casual indie strategy early
action adventure indie
action casual indie early
action casual indie early
action adventure casual indie
action indie multiplayer
action
adventure casual indie
action
action free rpg
adventure
adventure free indie
action adventure casual indie simulation
action adventure casual indie
action adventure indie rpg
indie
rpg 

action adventure indie
action adventure casual free indie simulation strategy
action adventure rpg early
casual indie
action adventure casual indie strategy
action adventure casual indie early
adventure
casual indie rpg strategy
action indie early
action adventure casual indie
action adventure casual indie
casual indie simulation
adventure casual indie
casual indie
casual indie simulation
adventure indie
action adventure casual indie
casual indie
action indie early
casual indie
adventure indie
casual
action casual simulation
action adventure indie
indie
casual indie multiplayer early
casual indie rpg simulation strategy
action racing
casual indie
casual indie
rpg strategy
casual indie simulation
adventure casual indie simulation
action adventure indie rpg simulation early
adventure indie
action casual indie
casual
action indie
casual indie simulation
casual indie simulation
indie
indie
action adventure indie
action casual indie strategy
action indie
action indie
adventure
action casual

In [71]:
print(len(y_couple))

1497


In [139]:
def multi_genres_labels(y):
    # list qui contient toutes les combinaisons de genres
    label_type = []
    N = y.shape[0]
    for i in range(N):
        y_split = y[i].split(" ")
        # on trie par ordre alphabetique pour eviter de devoir tester toutes les permutations de mots
        y_split.sort()
        genres_sort = ""
        # on réassemble tous les genres en un seul string
        for j in range(len(y_split)):
            # si c'est le dernier élement on ne met pas d'espace après
            if(j == len(y_split) - 1):
                genres_sort += y_split[j]
            else:
                genres_sort += y_split[j] + " "
        # si cette combinaison de genre n'est pas déjà dans label_type on l'ajoute
        if(genres_sort not in label_type):
            label_type.append(genres_sort)
    # nouveaux labels entiers
    int_labels = []
    # la nouvelle liste de labels est de la forme: int_label[int, genres], exemple: [0, 'action adventure free']
    for i in range(len(label_type)):
        int_labels.append([i, label_type[i]])
    return np.array(int_labels)

In [140]:
int_labels = multi_genres_labels(y_save)

In [141]:
int_labels

array([['0', 'action'],
       ['1', 'action free'],
       ['2', 'action free strategy'],
       ...,
       ['1472', 'action early free indie multiplayer'],
       ['1473', 'action adventure early indie multiplayer rpg strategy'],
       ['1474',
        'action adventure casual free indie rpg simulation sports strategy']],
      dtype='<U118')

In [162]:
# renvoi la liste des labels contenus dans y, multi_genre doit être a True si y est une liste de label multi_genre
def get_labels(y, multi_genre=False):
    labels = []
    # list utilisée pour tester si un genre composé de plusieurs genres est déjà dans la labels
    multi_labels = []
    N = y.shape[0]
    for i in range(N):
        if(multi_genre):
            if(y[i, 0] not in multi_labels):
                labels.append(y[i])
                multi_labels.append(i)
        else:
            y_split = y[i].split(" ")
            for j in y_split:
                if(j not in labels):
                    labels.append(j)
    return np.array(labels)

In [1]:
u = get_labels(y_save)
print(u)

In [2]:
u = get_labels(int_labels, multi_genre=True)
print(u)

NameError: name 'get_labels' is not defined